In [184]:
# matplotlib rebuild
import matplotlib as mpl
#mpl.font_manager._rebuild()
# matplotlib 환경 설정
import matplotlib.pyplot as plt
import pandas as pd
import FinanceDataReader as fdr

plt.rcParams["font.family"] = 'nanummyeongjo'
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.formatter.useoffset"] = False
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["axes.formatter.limits"] = -10000, 10000

### 종목코드 데이터 불러오기

In [185]:
## 엑셀 --> 크롤링 방식으로 수정해야 함
jongmok = pd.read_excel(r'C:\Users\skxod\Desktop\dev\Stock-Investment\Jonmokcode.xls')
### 종목코드 6자리 숫자로 변경
jongmok['종목코드'] = jongmok['종목코드'].astype('str')
for j in range(len(jongmok)):
    thres = len(jongmok.loc[j,'종목코드'])
    if thres <6:
        jongmok.loc[j,'종목코드'] = '0'*(6-thres) + jongmok.loc[j,'종목코드']
jongmok.head(2)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,브랜드엑스코퍼레이션,337930,기타 정보 서비스업,"애슬레저의류, 생활/청결 용품",2020-08-13,12월,"강민준, 이수연",http://www.brandxcorp.com/,서울특별시
1,영림원소프트랩,060850,소프트웨어 개발 및 공급업,"ERP Package, 유지 관리 등",2020-08-12,12월,권영범,http://ksystem.co.kr,서울특별시


#### EnterJongmok 함수 : 회사이름을 입력받아 종목코드 데이터로부터 종목코드를 조회하는 함수
 - 회사 리스트를 입력받는 함수 정의
 - Enter 키 입력하면 입력종료

In [186]:
def EnterJongmok(jongmok):
    ComList,StockList = [],[]
    cnt = 1
    while(1):
        txt = input(str(cnt)+'번째 기업명을 입력하세요. ')
        if txt=='':
            break
        else:
            cnt+=1
            ComList.append(txt)
    for c in ComList:
        StockList.append([c,jongmok.loc[jongmok['회사명']==c,'종목코드'].values[0]])
    return StockList       

#### EnterJongmok 함수 가지고 회사명과 종목코드 매칭시키기 

In [187]:
# EnterJongmok 함수 가지고 기업이름을 입력받아 코드를 조회하는 함수
stock_list = EnterJongmok(jongmok)
stock_list

1번째 기업명을 입력하세요. 삼성전자
2번째 기업명을 입력하세요. NAVER
3번째 기업명을 입력하세요. 카카오
4번째 기업명을 입력하세요. LG전자
5번째 기업명을 입력하세요. 


[['삼성전자', '005930'],
 ['NAVER', '035420'],
 ['카카오', '035720'],
 ['LG전자', '066570']]

#### financeDataReader 함수를 사용해서 주가 데이터 불러오기

In [188]:
#financeDataReader 함수를 사용해 주가 데이터 불러오기 
df_list = [fdr.DataReader(code, '2019-01-01', '2019-12-31')['Close'] for name, code in stock_list]

In [196]:
df = pd.concat(df_list, axis=1)
df.columns = [name for name, code in stock_list] 
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 246 entries, 2019-01-02 to 2019-12-30
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   삼성전자    246 non-null    int64
 1   NAVER   246 non-null    int64
 2   카카오     246 non-null    int64
 3   LG전자    246 non-null    int64
dtypes: int64(4)
memory usage: 19.6 KB


#### CalProfitRatio 함수
 - fdr 주가 데이터와 매수일과 매도일을 입력받아 기업별로 수익을 계산해주는 함수  
 - 계산방식  
   (매도일의 매도가-매수일의 매수가) / (매수일의 매수가)
   
     ※ 가정
  - 매수일에 매수해서 추가매수 없음
  - 매도일 전에 별도 매도 없음
  
  $\frac{Price_{sell} - Price_{buy}}{Price_{buy}}$
  

In [191]:
def CalProfitRatio(df,t_s,t_e):
    res = pd.DataFrame(round(df.apply(lambda x: round(x.loc[t_e]-x.loc[t_s])*100/x.loc[t_s]),2),columns=['수익률 (%)'])    
    return res    

In [195]:
CalProfitRatio(df,'2019-03-18','2019-04-24')

,수익률 (%)
삼성전자,2.40
NAVER,-8.33
카카오,9.77
LG전자,-5.75


## 이후 해야할 것  
 1. 회사명과 종목코드가 포함된 데이터를 엑셀에서 불러오는 것이 아니라 한국거래소 같은 사이트에서 크롤링으로 최신데이터를 받아올 수 있게끔 수정    
 2. PER,ROE,EPS,PBR 등 지표 데이터 크롤링 작업 필요(FinanceDataReader에서 PER,ROE 조회 기능이 있는지 봐야겠지만, 현재로서는 없는 거 같음)     
 3. 지표 데이터(PER,ROE,EPS)를 Python으로 불러와서 상위 몇 %에 드는 기업 리스트 정렬 기능 구현
 4. 수익계산에서 보완할 부분??